In [35]:
import numpy as np
import json


from rec2cal.utils import parse_ingredients, parse_instructions
from rec2cal.data_paths import rep_json, data_path

recipies = json.load(open(rep_json, "rb"))

def format_data(lst, partition="train"):
  dataset_X = []
  dataset_y = []
    
  for x in lst:
    if x["partition"] == partition:
      ingredients = parse_ingredients(x)
      instructions = parse_instructions(x["instructions"])
      calories = (x["nutr_values_per100g"]["energy"]/100) * sum(x['weight_per_ingr'])
      sentence = ingredients + ". " + instructions 

      dataset_X += [sentence + "\n\n###\n\n"]
      dataset_y += [" " + str(calories) + " END"]
  
  print(dataset_X[0])
  print(dataset_y[0])
  f_data = [{'prompt':i, 'completion':j} for i,j in zip(dataset_X, dataset_y)]
                    
  return f_data

stuff= format_data(recipies, "train")

8 ounce of yogurt-greek-plain-nonfat, 1 cup of strawberries-raw, 1/4 cup of cereals ready-to-eat-granola-homemade. Layer all ingredients in a serving dish.

###


 332.05964 END


In [ ]:
stuff

In [20]:
str_ = "\n".join(map(str,stuff)),

In [28]:
recipies = json.dump(stuff, open(data_path + "/open_ai.json", "w"))

In [34]:
import csv


data = [x.values() for x in stuff ]
with open(data_path + "/open_ai.csv", 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(("prompt", "completion"))

    # write multiple rows
    writer.writerows(data)

In [37]:
!openai tools fine_tunes.prepare_data -f ../data/open_ai.csv

Analyzing...


ERROR in read_any_format validator: File ../data/open_ai_fixed.csv does not exist.

Aborting...

In [ ]:
!openai api fine_tunes.create -t "../data/open_ai_prepared.jsonl" -m ada

In [25]:
len(str_[0])

27208403

In [26]:
str_[0][0:10]

"{'prompt':"